# PGT Trucking Capstone Project

Project student team: Peter Pan; Vincent Pan; Sanjit Sokhi, Jerry Wang, Jiadi Zhang

Advisor: Amr Farahat

Creation date: 2023-03-27

Choose the number of drivers and number of orders you want

In [225]:
num_driver =  20
num_order = 100
random_state_param = 886
time_limit = 1200
#mip_gap = 0.05  

 ## Importing packages

In [226]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import datetime, timedelta
import numpy as np
from geopy.geocoders import Nominatim 
from geopy import distance 
from multidict import MultiDict
import math
import gurobipy as gp
from gurobipy import GRB

 ## Setting functions

Distance functions to calculate the distance between two areas

In [227]:
def my_distance(origin_zip,dest_zip):
    geolocator = Nominatim(user_agent = "my_app_name")
    origin_loc = geolocator.geocode(origin_zip,country_codes ='us')
    dest_loc = geolocator.geocode(dest_zip,country_codes ='us')
    origin_latlon = (origin_loc.latitude,origin_loc.longitude)
    dest_latlon = (dest_loc.latitude,dest_loc.longitude)
    return (distance.distance(origin_latlon,dest_latlon).miles)


In [228]:
def haversine_distance(lon1, lat1, lon2, lat2):
    try:
        R = 6371  # radius of the Earth in kilometers
        dlon = math.radians(lon2 - lon1)
        dlat = math.radians(lat2 - lat1)
        a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        return R * c /1.609
    except:
        return None

Time function to calcualte the time consumption for a trucker to travel certain dsitance

In [229]:
def my_time(dis):
    speed = 7.6 * math.log(dis) + 4.5
    time_consumption = dis/speed
    return time_consumption

Function to standardize zip codes

In [230]:
def zip_correcter(zipcodes):
    zipdf = zipcodes.copy()
    corrected_zips = []
    for azip in zipdf:
        zip_length = len(str(azip))
        if zip_length < 5:
            corrected_zips.append((5-zip_length)*'0' + str(azip))
        elif zip_length > 5:
            corrected_zips.append(str(azip)[:5])
        else:
            corrected_zips.append(azip)
    return corrected_zips

 ## importing and processing data

Data import section

In [231]:
df = pd.read_csv("C:/Users/jiadiz/Desktop/PGT Trucking/initial model/Github/simulated_order_sample.csv",dtype={'dest_zipcode':str,'origin_zipcode':str})
df['dest_zipcode'] = zip_correcter(df['dest_zipcode'])
df['origin_zipcode'] = zip_correcter(df['origin_zipcode'])

In [232]:
df2 = df.dropna().sample(num_order, random_state = random_state_param)

In [233]:
df2['origin_zipcode'] = df2['origin_zipcode'].astype(str)
df2['dest_zipcode'] = df2['dest_zipcode'].astype(str)

In [234]:
samp_data.columns

Index(['ManifestNo', 'driverCode', 'ManifestLeg', 'OrderNo', 'NumberOfLegs',
       'PickupWindowStart', 'PickupWindowEnd', 'PickupArrive', 'PickupDepart',
       'DeliverWindowStart', 'DeliverWindowEnd', 'DeliverArrive',
       'DeliverDepart', 'CustomerNumber', 'ShipperCity', 'ShipperState',
       'ShipperZipCode', 'OriginCity', 'OriginState', 'ConsigneeCity',
       'ConsigneeState', 'ConsigneeCountry', 'ConsigneeZipCode',
       'DestinationCity', 'DestinationState', 'DispatchMiles', 'CarrierType',
       'LineHaulRevenue', 'CommodityCode', 'CommodityDesc', 'OrderWeight',
       'ManifestStatus', 'OrderCreateDate', 'WHTRACTOR', 'WHTRAILR1',
       'Window_length1', 'Window_length2', 'PickUpLength', 'DeliverTime',
       'DeliverStayTime'],
      dtype='object')

Creating pickup_midpoint

In [235]:
#Adding the columns pick_up_midpoint
from datetime import datetime
df2['pickup_start_date_time'] = df2.apply(lambda row: datetime.strptime(row['pickup_window_start'], '%m/%d/%Y %H:%M'),axis = 1)
df2['pickup_end_date_time'] = df2.apply(lambda row: datetime.strptime(row['pickup_window_end'], '%m/%d/%Y %H:%M'),axis = 1)
df2['pickup_midpoint'] = df2.apply(lambda row: row['pickup_start_date_time'] + (row['pickup_end_date_time'] - row['pickup_start_date_time']) / 2,axis = 1)

Creating Delivery_midpoint time

In [236]:
#Adding the columns delivery midpoints
df2['delivery_start_date_time'] = df2.apply(lambda row: datetime.strptime(row['delivery_window_start'], '%m/%d/%Y %H:%M'),axis = 1)
df2['delivery_end_date_time'] = df2.apply(lambda row: datetime.strptime(row['delivery_window_end'], '%m/%d/%Y %H:%M'),axis = 1)
df2['delivery_midpoint'] = df2.apply(lambda row: row['delivery_start_date_time'] + (row['delivery_end_date_time'] - row['delivery_start_date_time']) / 2,axis = 1)

Assuming that the drivers start working at 6am

In [237]:
mondays = []
import datetime
#Computing the mondays 6am of the weeks that the jobs are in
for ts in df2['pickup_midpoint']:
    # convert to datetime object
    dt = ts.to_pydatetime()

    # get the date and weekday
    date = dt.date()
    weekday = date.weekday()

    # calculate timedelta to previous Monday
    days_to_subtract = (7 + weekday) % 7
    monday = date - datetime.timedelta(days=days_to_subtract)

    # set hour and minute
    monday = datetime.datetime.combine(monday, datetime.time(hour=6, minute=0))

    # convert back to timestamp object
    monday = pd.Timestamp(monday)

    mondays.append(monday)

mondays  # output: [Timestamp('2023-03-06 06:00:00'),

df2['monday_6am'] = mondays

In [238]:
def time_to_arrive(mid_point):
    time_available = max(mid_point.weekday()-1, 0)*12 + mid_point.hour-6 + mid_point.minute/60
    return time_available
    

Computing whether the driver could arrive to the job if they begin driving at 6am

In [239]:
#computing the time difference between monday 6am and the pickup_midpoint time
df2['driver_home_to_job_midpoint_time_limit'] = df2.apply(lambda row : time_to_arrive(row['pickup_midpoint']),axis = 1)

Importing drivers

In [240]:
#cross merging driver sample and job dataframe
md = pd.read_csv('C:/Users/jiadiz/Desktop/PGT Trucking/initial model/Github/simulated_driver_sample.csv', dtype={'depot_zipcode':str})

In [241]:
md['depot_zipcode'] = zip_correcter(md['depot_zipcode'])
ds = md.sample(num_driver, random_state = random_state_param)
ds['key']=1
df2['key']=1

Creating terminals

In [242]:
#assigning terminals to drivers
terminals = []

for i in range (0,len(ds)):
    terminals.append(str('T'+str(i)))

ds['terminals'] = terminals

In [243]:
df2[df2['order_id'] == 3786037]

,index,order_id,origin_zipcode,dest_zipcode,pickup_window_start,pickup_window_end,delivery_window_start,delivery_window_end,LineHaulRevenue,DispatchMiles,hour_difference,pickup_start_date_time,pickup_end_date_time,pickup_midpoint,delivery_start_date_time,delivery_end_date_time,delivery_midpoint,monday_6am,driver_home_to_job_midpoint_time_limit,key


Merging drivers and orders to compute all driver-to-order possibilities

In [244]:
big_frame = pd.merge(ds,df2, on = 'key')

In [245]:
zips = pd.read_csv('uszips.csv', dtype={'zip':str})
zips['zip'] = zip_correcter(zips['zip'])
zips['zip'] = zips['zip'].astype(str)

Examine how many driver zip codes are not included in the zip code profile

In [246]:
len(zips['zip'][zips['zip'].isin(big_frame['depot_zipcode'])]) - len(big_frame['depot_zipcode'].unique())

0

In [247]:
big_frame['origin_zipcode'] = big_frame['origin_zipcode'].astype(str)
big_frame['dest_zipcode'] = big_frame['dest_zipcode'].astype(str)

Examine how many order zip codes are not included in the zip code profile

In [248]:
#zips['zip'][zips['zip'].isin(big_frame['origin_zipcode'])]
len(zips['zip'][zips['zip'].isin(big_frame['origin_zipcode'])]) - len(big_frame['origin_zipcode'].unique())

0

In [249]:
len(zips['zip'][zips['zip'].isin(big_frame['dest_zipcode'])]) - len(big_frame['dest_zipcode'].unique())

0

In [250]:
big_frame['dest_zipcode'][~big_frame['dest_zipcode'].isin(zips['zip'])].unique()

array([], dtype=object)

In [251]:
zips.columns = ['zip','depot_lat','depot_lng']
bigger_frame = pd.merge(zips,big_frame, left_on = 'zip', right_on = 'depot_zipcode')
zips.columns = ['zip','origin_lat','origin_lng']
bigger_bigger_frame = pd.merge(zips,bigger_frame, left_on = 'zip', right_on = 'origin_zipcode')
zips.columns = ['zip','dest_lat','dest_lng']
biggest_frame = pd.merge(zips, bigger_bigger_frame, left_on = 'zip', right_on = 'dest_zipcode')

Computing driver to job distance

In [252]:
biggest_frame['driver_to_job_distance'] = biggest_frame.apply(lambda row: haversine_distance(row['depot_lng'],row['depot_lat'],row['origin_lng'],row['origin_lat']),axis = 1)

In [253]:
distance_verification_sample = biggest_frame.sample(300)

Examining the validity of haversine distance. The examination costs time, so let the codes be commented out if not examining. 

In [254]:
import seaborn as sns

In [255]:
biggest_frame['haversine_distance'] = biggest_frame.apply(lambda row: haversine_distance(row['origin_lng'],row['origin_lat'],row['dest_lng'],row['dest_lat']),axis=1)

In [256]:
biggest_frame.columns

Index(['zip', 'dest_lat', 'dest_lng', 'zip_x', 'origin_lat', 'origin_lng',
       'zip_y', 'depot_lat', 'depot_lng', 'driver_id', 'depot_zipcode',
       'radius_origins', 'key', 'terminals', 'index', 'order_id',
       'origin_zipcode', 'dest_zipcode', 'pickup_window_start',
       'pickup_window_end', 'delivery_window_start', 'delivery_window_end',
       'LineHaulRevenue', 'DispatchMiles', 'hour_difference',
       'pickup_start_date_time', 'pickup_end_date_time', 'pickup_midpoint',
       'delivery_start_date_time', 'delivery_end_date_time',
       'delivery_midpoint', 'monday_6am',
       'driver_home_to_job_midpoint_time_limit', 'driver_to_job_distance',
       'haversine_distance'],
      dtype='object')

In [257]:
exam = pd.DataFrame((biggest_frame['haversine_distance'].drop_duplicates()-biggest_frame['DispatchMiles'].drop_duplicates())/biggest_frame['DispatchMiles'].drop_duplicates())

In [258]:
exam.dropna().mean()

0    38.306248
dtype: float64

In [259]:
biggest_frame['driver_to_job_distance_plus'] = biggest_frame['driver_to_job_distance'] + biggest_frame['haversine_distance']

In [260]:
biggest_frame['driver_to_job_time'] = biggest_frame['driver_to_job_distance_plus'] .apply(lambda row: my_time(row))

In [261]:
#all the feasible 1st jobs. If a job cannot be done on the first wave of driver's after 6am on Monday, the jobs cannot be done
first_wave = biggest_frame[biggest_frame['driver_to_job_time'] < biggest_frame['driver_home_to_job_midpoint_time_limit']]

In [262]:
first_wave = first_wave[first_wave['driver_to_job_distance']>0]
first_wave = first_wave.reset_index()


In [263]:
driver_to_order = {}
for index, row in first_wave.iterrows():
    key = (str(row['terminals']), str(row['order_id']))
    value = my_time(row['driver_to_job_distance_plus'])
    driver_to_order[key] = value

In [264]:
biggest_frame['job_to_driver_distance'] = biggest_frame.apply(lambda row: haversine_distance(row['origin_lng'], row['origin_lat'], row['depot_lng'], row['depot_lat']),axis = 1)

In [265]:
equal_day = []
for index, row in df2.iterrows():
    if row['delivery_start_date_time'].day != row['delivery_end_date_time'].day:
        equal_day.append('1')
    else:
        equal_day.append('0')


In [266]:
df2['same_day'] = equal_day

In [267]:
for index, row in df2[df2['same_day']=='1'].iterrows():
    print(row['delivery_start_date_time'], row['delivery_start_date_time'] + (row['delivery_end_date_time'] - row['delivery_start_date_time']) /2, row['delivery_end_date_time'])

2023-02-02 20:18:00 2023-02-03 01:15:30 2023-02-03 06:13:00
2023-02-05 22:21:00 2023-02-06 02:09:30 2023-02-06 05:58:00
2023-02-03 23:15:00 2023-02-04 09:26:00 2023-02-04 19:37:00
2023-02-03 21:04:00 2023-02-04 06:22:00 2023-02-04 15:40:00
2023-02-03 23:24:00 2023-02-04 06:09:30 2023-02-04 12:55:00
2023-02-04 23:02:00 2023-02-05 05:41:00 2023-02-05 12:20:00
2023-02-06 20:53:00 2023-02-07 04:49:30 2023-02-07 12:46:00
2023-02-06 18:37:00 2023-02-07 03:18:30 2023-02-07 12:00:00
2023-02-09 19:29:00 2023-02-10 00:44:00 2023-02-10 05:59:00
2023-02-03 21:18:00 2023-02-04 04:15:00 2023-02-04 11:12:00
2023-02-05 23:38:00 2023-02-06 04:24:30 2023-02-06 09:11:00
2023-02-07 07:18:00 2023-02-07 16:31:30 2023-02-08 01:45:00
2023-02-08 23:02:00 2023-02-09 02:59:00 2023-02-09 06:56:00
2023-02-04 18:15:00 2023-02-05 01:32:30 2023-02-05 08:50:00
2023-02-03 23:37:00 2023-02-04 02:43:00 2023-02-04 05:49:00
2023-02-07 15:17:00 2023-02-07 21:45:30 2023-02-08 04:14:00
2023-02-02 23:33:00 2023-02-03 04:18:30 

In [268]:
df2['delivery_midpoint'] = df2['delivery_start_date_time'] + (df2['delivery_end_date_time'] - df2['delivery_start_date_time']) /2

In [269]:
df2['pickup_midpoint'] = df2['pickup_start_date_time'] + (df2['pickup_end_date_time'] - df2['pickup_start_date_time']) /2

In [270]:
zips.columns = ['zip','origin_lat','origin_lng']
df2 = pd.merge(df2, zips, right_on = 'zip', left_on = 'origin_zipcode', how = 'left')
zips.columns = ['zip','dest_lat','dest_lng']
df2 = pd.merge(df2, zips, right_on = 'zip', left_on = 'dest_zipcode', how = 'left')

In [271]:
df2 = df2.drop_duplicates()

In [272]:
df2[df2['order_id'] == 3786037]

,index,order_id,origin_zipcode,dest_zipcode,pickup_window_start,pickup_window_end,delivery_window_start,delivery_window_end,LineHaulRevenue,DispatchMiles,...,monday_6am,driver_home_to_job_midpoint_time_limit,key,same_day,zip_x,origin_lat,origin_lng,zip_y,dest_lat,dest_lng


In [273]:
df2[df2['order_id'] == 3786037]

,index,order_id,origin_zipcode,dest_zipcode,pickup_window_start,pickup_window_end,delivery_window_start,delivery_window_end,LineHaulRevenue,DispatchMiles,...,monday_6am,driver_home_to_job_midpoint_time_limit,key,same_day,zip_x,origin_lat,origin_lng,zip_y,dest_lat,dest_lng


In [274]:
df3 = pd.merge(df2, df2, on = 'key')
df3.columns

Index(['index_x', 'order_id_x', 'origin_zipcode_x', 'dest_zipcode_x',
       'pickup_window_start_x', 'pickup_window_end_x',
       'delivery_window_start_x', 'delivery_window_end_x', 'LineHaulRevenue_x',
       'DispatchMiles_x', 'hour_difference_x', 'pickup_start_date_time_x',
       'pickup_end_date_time_x', 'pickup_midpoint_x',
       'delivery_start_date_time_x', 'delivery_end_date_time_x',
       'delivery_midpoint_x', 'monday_6am_x',
       'driver_home_to_job_midpoint_time_limit_x', 'key', 'same_day_x',
       'zip_x_x', 'origin_lat_x', 'origin_lng_x', 'zip_y_x', 'dest_lat_x',
       'dest_lng_x', 'index_y', 'order_id_y', 'origin_zipcode_y',
       'dest_zipcode_y', 'pickup_window_start_y', 'pickup_window_end_y',
       'delivery_window_start_y', 'delivery_window_end_y', 'LineHaulRevenue_y',
       'DispatchMiles_y', 'hour_difference_y', 'pickup_start_date_time_y',
       'pickup_end_date_time_y', 'pickup_midpoint_y',
       'delivery_start_date_time_y', 'delivery_end_date_time

In [275]:
df3['inter_job_distance'] = df3.apply(lambda row: haversine_distance(row['dest_lng_x'],row['dest_lat_x'],row['origin_lng_y'],row['origin_lat_y']), axis = 1)

In [276]:
df3 = df3[df3['inter_job_distance'] != 0]

In [277]:
# df3['inter_job_distance'].apply(lambda row: my_time(row))
inter_job_time = []
for index, row in df3.iterrows():
    inter_job_time.append(my_time(row['inter_job_distance']))
df3['inter_job_time'] = inter_job_time

In [278]:
df3['inter_job_time_limit'] = (df3['pickup_midpoint_y'] - df3['delivery_midpoint_x']).astype('timedelta64[h]') 

In [279]:
df4 = df3[df3['inter_job_time'] < df3['inter_job_time_limit']]

In [280]:
df4[['delivery_midpoint_x','pickup_midpoint_y','inter_job_time_limit']]

,delivery_midpoint_x,pickup_midpoint_y,inter_job_time_limit
100,2023-02-03 01:15:30,2023-02-05 08:26:00,55.0
102,2023-02-03 01:15:30,2023-02-05 04:30:30,51.0
104,2023-02-03 01:15:30,2023-02-05 01:04:30,47.0
112,2023-02-03 01:15:30,2023-02-04 09:01:00,31.0
114,2023-02-03 01:15:30,2023-02-03 20:06:30,18.0
...,...,...,...
9793,2023-02-04 05:13:00,2023-02-05 04:59:30,23.0
9798,2023-02-04 05:13:00,2023-02-05 17:27:30,36.0
9799,2023-02-04 05:13:00,2023-02-05 11:06:00,29.0
9970,2023-02-05 19:57:00,2023-02-06 15:39:00,19.0


 ## Creating arcs

In [281]:
import pickle

Purple arcs

In [284]:
# first_wave = first_wave.reset_index()

In [285]:
purple_out_arcs = {}
for index, row in first_wave.iterrows():
    key = (str(row['terminals']), str(row['order_id']))
    value = my_time(row['driver_to_job_distance_plus'])
    purple_out_arcs[key] = value

In [286]:
purple_out_flow_arcs = {}
for index, row in first_wave.iterrows():
    key = (str(row['driver_id']),str(row['terminals']), str(row['order_id']))
    value = my_time(row['driver_to_job_distance_plus'])
    purple_out_flow_arcs[key] = value

In [287]:
purple_in_arcs = {}

for i in range(0,len(first_wave)):
    purple_in_arcs[(str(str(first_wave['order_id'][i])),first_wave['terminals'][i])] = my_time(first_wave["driver_to_job_distance"][i])
    

In [288]:
purple_in_flow_arcs = {}

for index, row in first_wave.iterrows():
    key = (str(row['driver_id']), str(row['order_id']), str(row['terminals']))
    value = my_time(row["driver_to_job_distance"])
    purple_in_flow_arcs[key] = value



## Output arc segments

purple arcs

In [289]:
# with open('purple_in_arcs.pkl', 'wb') as f:
#     pickle.dump(purple_in_arcs, f)
# with open('purple_in_flow_arcs.pkl', 'wb') as f:
#     pickle.dump(purple_in_flow_arcs, f)

In [290]:
# with open('purple_out_arcs.pkl', 'wb') as f:
#     pickle.dump(purple_out_arcs, f)
# with open('purple_out_flow_arcs.pkl', 'wb') as f:
#     pickle.dump(purple_out_flow_arcs, f)

green arcs

In [291]:
df4 = df4.reset_index()

In [292]:
df4['second_job_distance'] = df4.apply(lambda row: haversine_distance(row['dest_lng_y'], row['dest_lat_y'], row['origin_lng_y'], row['origin_lat_y']),axis = 1)
df5 = df4[df4['second_job_distance'] > 0]
df5['second_job_time'] = df5['second_job_distance'].apply(lambda row: my_time(row))

In [293]:
df5['green_arc_time'] = df5['second_job_time']+df5['inter_job_time']

In [294]:
df5 = df5.reset_index()
green_arcs = {}

for i in range(0,len(df5)):
    green_arcs[str(df5['order_id_x'][i]),str(df5["order_id_y"][i])] = df5["green_arc_time"][i]

In [295]:
df6 = df5[['order_id_x','order_id_y','green_arc_time']]
df7 = first_wave[['driver_id','order_id']]

In [296]:
df8 = pd.merge(df6,df7,left_on = 'order_id_x', right_on = 'order_id')

In [297]:
df8 = df8.reset_index()
green_flow_arcs = {}

for i in range(0,len(df8 )):
    green_flow_arcs[str(df8['driver_id'][i]), str(df8['order_id_x'][i]),str(df8["order_id_y"][i])] = df8["green_arc_time"][i]

In [298]:
len(green_flow_arcs)

7139

In [299]:
# with open('green_flow_arcs.pkl', 'wb') as f:
#     pickle.dump(green_flow_arcs, f)

In [300]:
# with open('green_arcs.pkl', 'wb') as f:
#     pickle.dump(green_arcs, f)

terminals, drivers, and orders

In [301]:
# with open('terminals.pkl', 'wb') as f:
#     pickle.dump(terminals, f)

In [302]:
# with open('orders.pkl', 'wb') as f:
#     pickle.dump(orders, f)

In [303]:
#with open('driver_id.pkl', 'wb') as f:
#    pickle.dump(list(ds['driver_id']), f)

In [304]:
print(len(green_flow_arcs))
print(len(green_arcs))
print(len(purple_out_flow_arcs))
print(len(purple_out_arcs))
print(len(purple_in_arcs))
print(len(purple_in_flow_arcs))

7139
1163
1143
1143
1143
1143


In [305]:
all_arcs = {}
all_arcs.update(purple_out_arcs)
all_arcs.update(purple_in_arcs)
all_arcs.update(green_arcs)
all_arcs

{('T17', '1372'): 13.666101517847473,
 ('T14', '1372'): 19.134258987477743,
 ('T10', '1372'): 10.65942288373369,
 ('T13', '1372'): 15.130026007889965,
 ('T16', '1372'): 12.521917349927227,
 ('T4', '1372'): 18.870512297412102,
 ('T8', '1372'): 16.253179041520248,
 ('T3', '1372'): 20.574185395353673,
 ('T6', '1372'): 20.05551978828786,
 ('T1', '1372'): 15.410534163941106,
 ('T12', '1372'): 20.027565901303934,
 ('T15', '1372'): 20.86069493000413,
 ('T11', '1372'): 21.425969230439534,
 ('T17', '9420'): 26.876256897311027,
 ('T14', '9420'): 24.427315128317204,
 ('T10', '9420'): 20.28206078770492,
 ('T13', '9420'): 18.81618814294773,
 ('T16', '9420'): 18.418180202941926,
 ('T4', '9420'): 18.515204718575706,
 ('T8', '9420'): 17.38235422496566,
 ('T3', '9420'): 21.45904912981875,
 ('T6', '9420'): 21.035237176908186,
 ('T18', '9420'): 22.366781968081412,
 ('T1', '9420'): 15.96627780724933,
 ('T12', '9420'): 21.637481188843083,
 ('T15', '9420'): 22.02720565609134,
 ('T11', '9420'): 21.7117972365

In [306]:
flow_arcs = {}
flow_arcs.update(purple_out_flow_arcs)
flow_arcs.update(purple_in_flow_arcs)
flow_arcs.update(green_flow_arcs)

In [307]:
df = df.reset_index()
jobs_revenues = {}

for i in range(0,len(df)):
    jobs_revenues[str(df['order_id'][i])] = df["LineHaulRevenue"][i]

In [308]:
drivers = list(ds['driver_id'])
jobs = list(df2['order_id'].astype(str))
nodes = terminals + jobs

In [309]:
model_materials = (terminals, drivers,jobs, nodes, all_arcs, flow_arcs)

In [310]:
#df = df.reset_index()
#jobs_revenues = {}

#for i in range(0,len(df)):
#    jobs_revenues[str(df['order_id'][i])] = df["LineHaulRevenue"][i]
# model_materials = (terminals, drivers,jobs, nodes, all_arcs, flow_arcs)
# def sum_for_all_driver(d,flow_arcs):
#     out = []
#     for ch in flow_arcs:
#         if ch[0] == d:
#             out.append(ch)
#     return out
# def all_driver_value(x,d,flow_arcs):
#     out = []
#     for ch in flow_arcs:
        
#         if ch[0] == d:
            
#             out.append(flow_arcs[ch])
#     return out
# def find_the_next(path_example,k):
#     driver = k[0]
#     dest = k[2]
#     for ch in path_example:
#         if ch[0] == driver and dest == ch[1]:
#             return ch 
#     return 
# def generated_path(path_example):
#     all_path = []
#     path = ''
#     for ch in path_example:
#         driver = ch[0]
#         if ch[1][0]!='T':
#             continue 
    
#         ori = ch[1]
#         dest = ch[2]
#         path = driver +':'+ch[1] +'->' + ch[2] 
#         nex = find_the_next(path_example,ch)
#         path_example.remove(ch)
#         while nex:
#             path = path +'->' + nex[2]
#             temp = nex
#             path_example.remove(nex)
#             nex  = find_the_next(path_example,temp) 
            
#         all_path.append(path)
#         path = ''
        
#     return all_path 
# mod = gp.Model('model0')
# y = mod.addVars(jobs, vtype=GRB.BINARY, name= "y")
# x = mod.addVars(flow_arcs, vtype=GRB.BINARY, name= "x")
# obj_fn = mod.setObjective(gp.quicksum(y[j] for j in jobs), GRB.MAXIMIZE)

# single_driver = mod.addConstrs(x.sum('*', o, d) <= 1 for (o, d) in all_arcs)
# flow_conservation = mod.addConstrs(x.sum(d, '*', j) == x.sum(d, j, '*') for d in drivers for j in nodes if j not in terminals)
# served_jobs = mod.addConstrs(x.sum('*', '*', j)  == y[j] for j in jobs)
#terminal_constraint = mod.addConstrs((x.sum(d, '*', t) <=1  for d in drivers for t in terminals))
# for d in drivers:
#     driver =  sum_for_all_driver(d,flow_arcs)
    
# mod.addConstr(gp.quicksum(flow_arcs[ch]* x[ch] for ch in driver) <= 20)
# mod.optimize()

In [311]:
with open('model_materials.pkl', 'wb') as f:
    pickle.dump(model_materials, f)

In [312]:
def sum_for_all_driver(d,flow_arcs):
    out = []
    for ch in flow_arcs:
        if ch[0] == d:
            out.append(ch)
    return out

In [313]:
def all_driver_value(x,d,flow_arcs):
    out = []
    for ch in flow_arcs:
        
        if ch[0] == d:
            
            out.append(flow_arcs[ch])
    return out

In [314]:
def find_the_next(path_example,k):
    driver = k[0]
    dest = k[2]
    for ch in path_example:
        if ch[0] == driver and dest == ch[1]:
            return ch 
    return 

In [315]:
def generated_path(path_example):
    all_path = []
    path = ''
    for ch in path_example:
        driver = ch[0]
        if ch[1][0]!='T':
            continue 
    
        ori = ch[1]
        dest = ch[2]
        path = driver +':'+ch[1] +'->' + ch[2] 
        nex = find_the_next(path_example,ch)
        path_example.remove(ch)
        while nex:
            path = path +'->' + nex[2]
            temp = nex
            path_example.remove(nex)
            nex  = find_the_next(path_example,temp) 
            
        all_path.append(path)
        path = ''
        
    return all_path 

In [316]:
mod = gp.Model('model0')

In [317]:
y = mod.addVars(jobs, vtype=GRB.BINARY, name= "y")
x = mod.addVars(flow_arcs, vtype=GRB.BINARY, name= "x")

In [318]:
obj_fn = mod.setObjective(gp.quicksum(jobs_revenues[j]* y[j] for j in jobs), GRB.MAXIMIZE)

In [319]:
# obj_fn = mod.setObjective(gp.quicksum(y[j] for j in jobs), GRB.MAXIMIZE)

In [320]:
u = mod.addVars(nodes, vtype=GRB.INTEGER, name="u")

In [321]:
terminal_constraint = mod.addConstrs((x.sum(d, '*', t) <=1  for d in drivers for t in terminals))


In [322]:
flow_conservation = mod.addConstrs((x.sum(d, '*', j) == x.sum(d, j, '*') for d in drivers for j in nodes if j not in terminals),name="flow_conservation",
)

In [323]:
single_driver = mod.addConstrs(x.sum('*', o, d) <= 1 for (o, d) in all_arcs)

In [324]:
served_jobs = mod.addConstrs(x.sum('*', '*', j)  == y[j] for j in jobs)

In [325]:
mip_gap = 0.05  # 5% optimality gap

# mod.setParam(GRB.Param.MIPGap, mip_gap)
mod.setParam(GRB.Param.TimeLimit, time_limit)

Set parameter TimeLimit to value 1200


In [326]:
for d in drivers:
    driver =  sum_for_all_driver(d,flow_arcs)
    
    mod.addConstr(gp.quicksum(flow_arcs[ch]* x[ch] for ch in driver) <= 80)

In [327]:
mod.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700HQ CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5969 rows, 9645 columns and 44939 nonzeros
Model fingerprint: 0x198e94ee
Variable types: 0 continuous, 9645 integer (9525 binary)
Coefficient statistics:
  Matrix range     [5e-01, 9e+01]
  Objective range  [2e+02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 5241 rows and 3483 columns
Presolve time: 0.64s
Presolved: 728 rows, 6162 columns, 23180 nonzeros
Variable types: 0 continuous, 6162 integer (6162 binary)
Found heuristic solution: objective 54922.136867

Root relaxation: objective 1.535999e+05, 2728 iterations, 0.15 seconds (0.09 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf |

In [328]:
compelete_job = []
for i in flow_arcs:
    if (x[i].x >=0.99):
        compelete_job.append(i)

In [330]:
df[df['order_id'].astype(str).isin(['3779312','3783956','3785314'])]

,level_0,index,order_id,origin_zipcode,dest_zipcode,pickup_window_start,pickup_window_end,delivery_window_start,delivery_window_end,LineHaulRevenue,DispatchMiles,hour_difference


In [331]:
final_result = generated_path(compelete_job)
while compelete_job != []:
    final_result+= generated_path(compelete_job)
final_result

['Driver41:T17->9420->5071->T17',
 'Driver39:T10->1540->5800->T10',
 'Driver458:T3->8965->9005->8584->T3',
 'Driver12:T18->3877->7226->T18',
 'Driver504:T6->9245->7997->T6',
 'Driver24:T7->476->8792->T7',
 'Driver477:T19->7616->9556->T19',
 'Driver181:T12->9119->6624->T12',
 'Driver311:T2->405->9067->8455->T2',
 'Driver457:T11->3896->2381->T11',
 'Driver294:T13->45->7577->3305->T13',
 'Driver286:T0->5004->6585->T0',
 'Driver453:T14->3757->6329->4229->T14',
 'Driver361:T16->4204->5593->8808->T16',
 'Driver82:T9->111->3398->T9',
 'Driver378:T4->765->8170->T4',
 'Driver550:T8->3925->1987->T8',
 'Driver390:T15->7734->6439->T15',
 'Driver223:T1->9701->2013->T1',
 'Driver247:T5->8431->2274->T5']

In [332]:
selected_jobs_count= 0
for i in jobs:
    #if (x[i].x >= 0.99):
        #print(x[i].x)
        if y[i].x > 0.5:
           selected_jobs_count += 1 

In [333]:
print('selected_jobs_count :' +str(selected_jobs_count), 'job_count_in_job_pool :' + str(len(df2)), 'job_selection_rate :' + str(selected_jobs_count/len(df2)))

selected_jobs_count :45 job_count_in_job_pool :100 job_selection_rate :0.45


In [334]:
print('optimized_revenue :' +str(mod.ObjVal), 'revenue_in_pool:' + str(df2['LineHaulRevenue'].sum()), 'optimized_revenue_to_pool_revenue_rate :' + str(mod.ObjVal/df2['LineHaulRevenue'].sum()))

optimized_revenue :152205.7261814886 revenue_in_pool:293883.3404569817 optimized_revenue_to_pool_revenue_rate :0.517912059747287
